export KEY_MYSQL='1533542415'
export API_KEY_YELP="Ln6gcw24qn2x3RFJn0D89tGTBCRfrrpRrXqEp7y8fsaDFd2yjf7byiYcDirTF0VNr-bgsPE_kZPRqY0mpv7pXVd5KuVpzQC8cX5NzywiwBH1djTKvzQqNumm83JkZXYx"

In [4]:
import pandas as pd
%load_ext memory_profiler

from mysql_connection import *
from transform_API import *
import pymysql as mysql
import pandas as pd
from dotenv import load_dotenv 
import os


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [17]:
def yelp_ER():
    
    """
        Esta funcion realiza el proceso de ETL completo respecto de la API de yelp para los restaurantes en la base datos mysql.
        Para esto aplica las funciones:
            * extract_businesses
            * transform_business
            * get_table
            * mysql_get_connection
            * get_categories
    
    """
    
    yelp_new_data = pd.read_parquet('./datalake/business_trasnform.parquet') # Realizo las trasnformaciones necesarias para que los datos esten limpios

    yelp_origen = get_table('business_yelp') # Cargo de la base de datos la tabla de yelp en un dataframe
    
    yelp_new_data = yelp_new_data[~(yelp_new_data['business_id'].isin(yelp_origen['business_id'].unique().tolist()))] #De los restaurantes extraidos tomo solo los que su id NO esta en la DB
    
    return yelp_origen
    
    print(f'La cantidad de restaurantes a ingestar es {yelp_new_data.shape[0]}')
    
    if yelp_new_data.shape[0] != 0:
        conexion = get_connection_mysql() # Genero una conexion a mysql
        cursor = conexion.cursor() 
        
        consulta = "INSERT INTO business_yelp  VALUES(%s,%s,%s,%s,%s,%s)" 
        yelp_insert = yelp_new_data[['business_id','name','latitude','longitude','stars','state_id']].copy()
        cursor.executemany(consulta,yelp_insert.values.tolist() ) # Inserto los nuevos locales, sin insertar las categorias
        print(f'{yelp_insert.shape[0]} nuevos negocios')
        conexion.commit()
        conexion.close()
        
        categories_origen = get_table('categories') # Cargo la tabla de categorias de la base de datos.    
        max_id = categories_origen['categories_id']
        categorias_new_data = get_categories(yelp_new_data.copy())
        
        categorias_new_data['categories'] = categorias_new_data['categories'].apply(lambda x: x.lower())

        #Agrego la categoria Restaurants a cada local
        '''
        df_restaurant = categorias_new_data.drop_duplicates(subset='business_id').copy()
        df_restaurants = categorias_new_data.drop_duplicates(subset='business_id').copy()
        
        df_restaurant['categories'] = 'restaurants'
        df_restaurants['categories'] = 'restaurants'
        categorias_new_data = pd.concat([categorias_new_data,df_restaurant])
        categorias_new_data = pd.concat([categorias_new_data,df_restaurants])
        print(categorias_new_data.columns)
        
        
        categorias_new = categorias_new_data[~(categorias_new_data['categories'].isin(categories_origen['name']))] # Selecciono las categorias que no estan en la DB
        categorias_new['categories_id'] = list(range(max_id + 1, max_id + 1 + categorias_new.shape[0]))

        categories = categorias_new.drop_duplicates(subset='categories')['categories'].copy() # Elimino las categorias duplicadas y las convierto en lista de listas.
        conexion = get_connection_mysql() 
        cursor = conexion.cursor()
        
        # Ingesto las nuevas categorias.
        consulta = "INSERT INTO categories  VALUES(%s,%s)"
        cursor.executemany(consulta, categories.values.tolist())
        print(f'{categories.shape[0]} nuevas categorias ingestadas')
        conexion.commit()
        conexion.close()
        
        
        categories_acualizada = get_table('categories') # Cargo la tabla de categorias actualizada.
        
        #Hago un join entre la tabla business_id,categoria creada anteriormente con las categorias de la BD, y me quedo solo con business_id y categoria id
        categorias_yelp_new =  pd.merge(categories_acualizada,categorias_new_data,left_on='name',right_on='categories',how='inner')
        
        conexion = get_connection_mysql()
        
        # Como business id ya es unico simplemente agrego las filas a la tabla cateogires_yelp
        try:
            cursor = conexion.cursor()
            consulta = "INSERT INTO categories_yelp  VALUES(%s,%s)"
            cursor.executemany(consulta, categorias_yelp_new[['business_id','categories_id']].values.tolist())
            conexion.commit()
            conexion.close()
        except Exception as e:
            print(f"Error al ejecutar la consulta SQL: {e}")
            # Aquí puedes agregar código adicional para manejar la excepción según tus necesidades.
            # Por ejemplo, podrías hacer un rollback si es necesario.
        finally:
            # Este bloque se ejecutará siempre, asegurando que la conexión se cierre incluso en caso de excepción.
            if conexion and conexion.open:
                conexion.rollback()  # Hacer un rollback en caso de excepción antes de cerrar la conexión.
                conexion.close()
    else:
        return 'No habian restaurantes para ingestar'


'''
    

    
#------------------------------------------------------------------------      
#----------------------------REVIEWS-------------------------------------    
#------------------------------------------------------------------------  


yelp_ER()

,business_id,name,latitude,longitude,avg_stars,state_id
0,__2LkeVwTuKar5AE95Idzw,McDonalds,27.8931988,-82.3198443,1.58,1
1,__Dbc84VjNmVLNgyTUP4iw,Vampire Penguin,27.8411655,-82.7883552,4.70,1
2,_-BFOIdAdk-1ejLSPmaTsQ,The Hungry Greek,27.9600887,-82.7275122,3.97,1
3,_-SSObHzljeA11HAF0lIrQ,Hardees,27.9955205,-82.5124740,2.45,1
4,_05zwVNkOSa9Kym6-omCnQ,Gordis Restaurant,28.0687238,-82.4412916,3.20,1
...,...,...,...,...,...,...
13182,ZzqqkL9mHsOnFHJvVd-10Q,Wawa,39.9275149,-74.9534366,3.95,4
13183,ZZQzt65614oon6yfokjRIA,Bon Zanmi Haitian Cuisine,28.0806110,-82.4510520,4.37,1
13184,zZrDoiQIUmiVkifJx0h_KA,Old Town Coffee Downtown,34.4230563,-119.7045147,4.38,2
13185,zZwnm2htQGTBJkpGy2hemw,Popeyes Louisiana Kitchen,39.8044850,-75.0251300,2.18,4


In [12]:
codigo = yelp_ER()

yelp_new_data = pd.read_parquet('./datalake/business_trasnform.parquet')

yelp_new_data.loc[yelp_new_data['business_id'].isin(codigo['business_id'].unique().tolist())]

,business_id,name,latitude,longitude,categories,stars,state_id
0,qjnpkS8yZO8xcyEIy5OU9A,Girl & The Goat,41.884181,-87.647947,"[New American, Bakeries, Coffee & Tea]",4.5,3
1,boE4Ahsssqic7o5wQLI04w,The Purple Pig,41.890694,-87.624782,"[Tapas/Small Plates, Mediterranean, New American]",4.5,3
2,xoi7Cw7FoknAx5p880RtWQ,Au Cheval,41.884660,-87.647668,"[Bars, American]",4.5,3
3,y8jVBaHh8ntYkVqDXpsi1Q,Cafe Ba-Ba-Reeba!,41.918984,-87.648687,"[Spanish, Tapas Bars, Venues & Event Spaces]",4.5,3
4,9nUrDQ_REhU6sgcgXFAyfA,Aba,41.886944,-87.648795,"[Mediterranean, Tapas/Small Plates]",4.5,3
...,...,...,...,...,...,...,...
45,gsJYTks9w2BQOsZnJNFnwA,Austria Hof Restaurant,37.647592,-119.000261,"[German, Bars, New American]",4.0,2
46,UB_Z52wC2-BlUqd0xDtddg,The Berrock Shop,36.823576,-119.826896,"[Specialty Food, Burgers, Sandwiches]",4.5,2
47,VBKv8Q7DlT64guAZi4bxJw,Max's Bistro and Bar,36.823521,-119.825943,"[Seafood, Steakhouses, New American]",4.0,2
48,RxBjn1XKhSTeTZ7bqMVysA,Sam's Italian Deli & Market,36.772659,-119.772999,"[Delis, Beer, Wine & Spirits, International Gr...",4.5,2


In [15]:
len(codigo['business_id'].unique().tolist())

13187

In [5]:
def transform_business():
    
    """Esta función realiza trasnformaciones sobre los restaurantes a partir del dataset tomando de la API.

    Args:
        yelp_bussines(pd.DataFrame): Data Frame de la API.
    Returns:
        pd.DataFrame: DataFrame restaurantes.
    """
    
    yelp_bussines = pd.read_parquet('./datalake/business_API.parquet')
    
    
    if yelp_bussines is not None and 'categories' in yelp_bussines.columns:
        yelp_bussines['categories'] = yelp_bussines['categories'].apply(lambda x: [item['title'] for item in x] if isinstance(x, list) else [])
        yelp_bussines['location.state'] = yelp_bussines['location.state'].apply(state_normalize)

        yelp_bussines.rename(columns={
        'id':'business_id',
        'name':'name',
        'coordinates.latitude':'latitude',
        'coordinates.longitude':'longitude',
        'categories':'categories',
        'rating':'stars',
        'location.state':'state_id',
        
        },inplace=True)
        columnas = ['business_id', 'name', 'latitude','longitude','categories','stars','state_id']
        yelp_bussines = yelp_bussines[columnas]
        
        yelp_bussines.loc[:, 'latitude'] = round(yelp_bussines['latitude'], 8)
        yelp_bussines.loc[:, 'longitude'] = round(yelp_bussines['longitude'], 8)
        yelp_bussines.loc[:, 'stars'] = round(yelp_bussines['stars'], 2)
        
        
        #return yelp_bussines.to_parquet('./datalake/business_trasnform.parquet')
        return yelp_bussines
    else:
        return 'No se cargaron nuevos datos.'
    
    
transform_business()

,business_id,name,latitude,longitude,categories,stars,state_id
0,qjnpkS8yZO8xcyEIy5OU9A,Girl & The Goat,41.884181,-87.647947,[],4.5,3
1,boE4Ahsssqic7o5wQLI04w,The Purple Pig,41.890694,-87.624782,[],4.5,3
2,xoi7Cw7FoknAx5p880RtWQ,Au Cheval,41.884660,-87.647668,[],4.5,3
3,cKZNbMvoqJaUe7n6lf6i7w,Wildberry Pancakes and Cafe,41.884668,-87.622880,[],4.5,3
4,y8jVBaHh8ntYkVqDXpsi1Q,Cafe Ba-Ba-Reeba!,41.918984,-87.648687,[],4.5,3
...,...,...,...,...,...,...,...
45,M7dZODPoiOAJHdKzIWMPHw,Morrison's Restaurant and Bar,37.649010,-118.974250,[],4.0,2
46,VvPOfhbH6BH3I-vFG1hALQ,Ramen Hayashi,36.837882,-119.685182,[],4.5,2
47,PGUM86R57lXPi4LFHJWhAg,Dish Bistro,37.637530,-118.966839,[],5.0,2
48,3uBgSXc_FP1sRSKfjWExoQ,The Curry Pizza Company,36.808120,-119.848220,[],4.5,2


In [7]:
import pandas as pd

def transform_business():
    """Esta función realiza transformaciones sobre los restaurantes a partir del dataset tomado de la API.

    Args:
        None
    Returns:
        pd.DataFrame: DataFrame de restaurantes transformado.
    """
    try:
        yelp_business = pd.read_parquet('./datalake/business_API.parquet')
        
        if 'categories' in yelp_business.columns and not yelp_business['categories'].empty:
            yelp_business['categories'] = yelp_business['categories'].apply(lambda x: [item['title'] for item in x] if isinstance(x, list) else [])
            yelp_business['location.state'] = yelp_business['location.state'].apply(state_normalize)

            yelp_business.rename(columns={
                'id': 'business_id',
                'name': 'name',
                'coordinates.latitude': 'latitude',
                'coordinates.longitude': 'longitude',
                'categories': 'categories',
                'rating': 'stars',
                'location.state': 'state_id',
            }, inplace=True)
            
            columns = ['business_id', 'name', 'latitude', 'longitude', 'categories', 'stars', 'state_id']
            yelp_business = yelp_business[columns]
            
            yelp_business.loc[:, 'latitude'] = round(yelp_business['latitude'], 8)
            yelp_business.loc[:, 'longitude'] = round(yelp_business['longitude'], 8)
            yelp_business.loc[:, 'stars'] = round(yelp_business['stars'], 2)

            return yelp_business
        else:
            return 'No se cargaron nuevos datos o la columna "categories" está vacía.'
    except Exception as e:
        return f'Error: {str(e)}'


transform_business()

,business_id,name,latitude,longitude,categories,stars,state_id
0,qjnpkS8yZO8xcyEIy5OU9A,Girl & The Goat,41.884181,-87.647947,[],4.5,3
1,boE4Ahsssqic7o5wQLI04w,The Purple Pig,41.890694,-87.624782,[],4.5,3
2,xoi7Cw7FoknAx5p880RtWQ,Au Cheval,41.884660,-87.647668,[],4.5,3
3,cKZNbMvoqJaUe7n6lf6i7w,Wildberry Pancakes and Cafe,41.884668,-87.622880,[],4.5,3
4,y8jVBaHh8ntYkVqDXpsi1Q,Cafe Ba-Ba-Reeba!,41.918984,-87.648687,[],4.5,3
...,...,...,...,...,...,...,...
45,M7dZODPoiOAJHdKzIWMPHw,Morrison's Restaurant and Bar,37.649010,-118.974250,[],4.0,2
46,VvPOfhbH6BH3I-vFG1hALQ,Ramen Hayashi,36.837882,-119.685182,[],4.5,2
47,PGUM86R57lXPi4LFHJWhAg,Dish Bistro,37.637530,-118.966839,[],5.0,2
48,3uBgSXc_FP1sRSKfjWExoQ,The Curry Pizza Company,36.808120,-119.848220,[],4.5,2


In [15]:
def yelp_review_ER():
    """
        Esta funcion realiza el proceso de ETL completo respecto de la API de yelp para las reviews de restaurantes y las sube en la base datos mysql.
        Para esto aplica las funciones:
            * extract_businesses
            * transform_business
            * get_table
            * mysql_get_connection
    
    """
    review_new_data = pd.read_parquet('./datalake/reviews_yelp_transform.parquet') # Hago las trasnformaciones sobre el dataframe.
    
    reviews_yelp_origen = get_review_yelp('reviews_yelp') # Consulto la tabla las ultimas resenas

    users_old = get_filtered_table('user_yelp', review_new_data['user_id'].unique().tolist())
    
    #Filtro solo las reviews donde su columna date sea mayor a la maxima existente en la base de datos.
    #print(f'{review_new_data.shape[0]} reviews a ingestar')

    review_new_data = review_new_data[(~review_new_data['review_id'].isin(reviews_yelp_origen['review_id']))]
    
    
    #### USERS ####  
    
    
    users = review_new_data.groupby('user_id').agg({
        'name':'first',
        'date':'min',
        'review_id':'count',
        'stars':'mean'
        
    }).reset_index().rename(columns={'date': 'creation','review_id':'review_count'})
    
    #### DATAFRAME CON LOS USUARIOS NUEVOS
    new_users = users[~(users['user_id'].isin(users_old['user_id']))]
    new_users['influence'] = 0

    
    
     #### DATAFRAME CON LOS USUARIOS EXISTENTES
    #exist_user = users[(users['user_id'].isin(users_old['user_id']))] # Usuarios existentes
    
    exist_user = pd.concat([users_old, users[users['user_id'].isin(users_old['user_id'])]]) # Hago un merge de los usuarios de la BD y la llegada
    # Para los nuevos usuarios encuentro la nueva review_count y stars
    exist_user['stars'] = pd.to_numeric(exist_user['stars'], errors='coerce')

    exist_user = exist_user.groupby('user_id').agg({
        'name':'first',
        'creation':'first',
        'review_count':'count',
        'stars':'mean'
        
    }).reset_index()
    

    
    new_users_ids = new_users['user_id'].tolist()  # Obtener IDs de nuevos usuarios
    existing_users_ids = users_old['user_id'].tolist()  # Obtener IDs de usuarios existentes

    new_users_to_insert = new_users[new_users['user_id'].isin(existing_users_ids) == False]

    
    
    
    
    # Verificar si hay usuarios nuevos para insertar
    if not new_users_to_insert.empty:
        try:
            conexion = get_connection_mysql()
            cursor = conexion.cursor()

            # Insertar nuevos usuarios en la tabla user_yelp
            consulta_new_user = (
                'INSERT INTO user_yelp VALUES (%s, %s, %s, %s, %s, %s)'
            )
            cursor.executemany(consulta_new_user, new_users_to_insert[['user_id', 'name', 'creation', 'review_count', 'influence', 'stars']].values.tolist())
            print(f'{new_users_to_insert.shape[0]} nuevos usuarios cargados.')

            conexion.commit()
        except Exception as e:
            print(f"Error al insertar nuevos usuarios: {e}")
            if conexion and conexion.open:
                conexion.rollback()
        finally:
            if conexion and conexion.open:
                conexion.close()
            
    
    #### REVIEWS #####
    
    
    #print("Datos a insertar:", review_new_data.drop(columns=['name']).columns)
    print(review_new_data)
    try:
        conexion= get_connection_mysql()
        cursor = conexion.cursor()
        consulta = "INSERT INTO reviews_yelp  VALUES(%s,%s,%s,%s,%s,%s)"
        cursor.executemany(consulta,review_new_data.drop(columns=['name']).values.tolist() )
        print(f'{review_new_data.shape[0]} nuevas reviews')
        conexion.commit()
        conexion.close()
    except Exception as e:
        print(f"Error al ejecutar la consulta SQL: {e}")
        # Aquí puedes agregar código adicional para manejar la excepción según tus necesidades.
        # Por ejemplo, podrías hacer un rollback si es necesario.
    finally:
        # Este bloque se ejecutará siempre, asegurando que la conexión se cierre incluso en caso de excepción.
        if conexion and conexion.open:
            conexion.rollback()  # Hacer un rollback en caso de excepción antes de cerrar la conexión.
            conexion.close()







In [ ]:
def yelp_review_ER():
    """
        Esta funcion realiza el proceso de ETL completo respecto de la API de yelp para las reviews de restaurantes y las sube en la base datos mysql.
        Para esto aplica las funciones:
            * extract_businesses
            * transform_business
            * get_table
            * mysql_get_connection
    
    """
    
    review_new_data = pd.read_parquet('./datalake/reviews_yelp_transform.parquet') # Hago las trasnformaciones sobre el dataframe.
    
    reviews_yelp_origen = get_review_yelp('reviews_yelp') # Consulto la tabla las ultimas resenas

    users_old = get_filtered_table('user_yelp', review_new_data['user_id'].unique().tolist())
    


    print(users_old.shape[0])
    #Filtro solo las reviews donde su columna date sea mayor a la maxima existente en la base de datos.
    print(f'{review_new_data.shape[0]} reviews a ingestar')
    
    review_new_data = review_new_data[(~review_new_data['review_id'].isin(reviews_yelp_origen['review_id']))]
    review_new_data['date'] = pd.to_datetime(review_new_data['date'])

    
    #### USERS ####  
    print(review_new_data.info())

    grouped_data = review_new_data.groupby('user_id') # Rrealizo el agrupamiento por usuarios y calculas las metricas.
    min_dates = grouped_data['date'].min()
    first_names = grouped_data['name'].first()
    review_counts = grouped_data['review_id'].count()
    mean_stars = grouped_data['stars'].mean()
    users = pd.DataFrame({
        'name': first_names,
        'creation': min_dates,
        'review_count': review_counts,
        'stars': mean_stars
    }).reset_index()
    users['influence'] = 0
    users = users[['user_id','name','creation','review_count','influence','stars']]
    #### DATAFRAME CON LOS USUARIOS NUEVOS
    new_users = users[~(users['user_id'].isin(users_old['user_id']))]
    new_users['influence'] = 0
    
    
     #### DATAFRAME CON LOS USUARIOS EXISTENTES
    #exist_user = users[(users['user_id'].isin(users_old['user_id']))] # Usuarios existentes
    
    
    exist_user = pd.concat([users_old, users[users['user_id'].isin(users_old['user_id'])]]) # Hago un merge de los usuarios de la BD y la llegada
    # Para los nuevos usuarios encuentro la nueva review_count y stars
    exist_user['stars'] = exist_user['stars'].astype('float')
    # Realizo el agrupamiento por usuarios existentes y calculo las metricas.
    grouped_data = exist_user.groupby('user_id')
    min_dates = grouped_data['creation'].min()
    first_names = grouped_data['name'].first()
    review_counts = grouped_data['review_count'].count()
    mean_stars = grouped_data['stars'].mean()
    exist_user = pd.DataFrame({
        'name': first_names,
        'creation': min_dates,
        'review_count': review_counts,
        'stars': mean_stars
    }).reset_index()
    
    print(exist_user)
    
    if not exist_user.empty:
        try:
            conexion = get_connection_mysql()
            cursor = conexion.cursor()
            
            consulta_new_user = (
            'INSERT INTO user_yelp  VALUES(%s,%s,%s,%s,%s,%s)'
                )

            # Ejecutar la consulta con execumany.
            cursor.executemany(consulta_new_user, new_users[['user_id', 'name','creation', 'review_count', 'influence','stars']].values.tolist())
            print(f'{new_users.shape[0]} usuarios nuevos cargados.')
            # Consulta de actualización con placeholders.
            consulta_old_user = (
                "UPDATE user_yelp "
                "SET name = %s, creation = %s, review_count = %s, stars = %s "
                "WHERE user_id = %s"
            )

            # Ejecutar la consulta con execumany.
            cursor.executemany(consulta_old_user, exist_user[['name', 'creation', 'review_count', 'stars', 'user_id']].values.tolist())
            print(f'{exist_user.shape[0]} usuarios actualizados.')
            conexion.commit()
        except Exception as e:
            print(f"Error al ejecutar la consulta SQL: {e}")
            # Maneja la excepción según tus necesidades.
            # Puedes hacer un rollback si es necesario.
        finally:
            # Cierra la conexión en el bloque finally para asegurar que se cierre incluso en caso de excepción.
            if conexion and conexion.open:
                conexion.rollback()
                conexion.close()
            
    print(review_new_data.shape[0])
    
    #### REVIEWS #####
    
    
    #print("Datos a insertar:", review_new_data.drop(columns=['name']).columns)
    print(review_new_data)
    try:
        conexion= get_connection_mysql()
        cursor = conexion.cursor()
        consulta = "INSERT INTO reviews_yelp  VALUES(%s,%s,%s,%s,%s,%s)"
        cursor.executemany(consulta,review_new_data.drop(columns=['name']).values.tolist() )
        print(f'{review_new_data.shape[0]} nuevas reviews')
        conexion.commit()
        conexion.close()
    except Exception as e:
        print(f"Error al ejecutar la consulta SQL: {e}")
        # Aquí puedes agregar código adicional para manejar la excepción según tus necesidades.
        # Por ejemplo, podrías hacer un rollback si es necesario.
    finally:
        # Este bloque se ejecutará siempre, asegurando que la conexión se cierre incluso en caso de excepción.
        if conexion and conexion.open:
            conexion.rollback()  # Hacer un rollback en caso de excepción antes de cerrar la conexión.
            conexion.close()